In [10]:
!cat .env

PROJECT_ID=winged-quanta-472908-n1
LOCATION=us-central1
BUCKET_URI=gs://oppe1_winged-quanta-472908-n1
BUCKET_NAME=oppe1_winged-quanta-472908-n1


In [11]:
import mlflow
from mlflow import MlflowClient
from mlflow.models import infer_signature
from pprint import pprint
import requests

response = requests.get('https://api.ipify.org')

mlflow.set_tracking_uri(f"http://{response.text}:8100")
client = MlflowClient(mlflow.get_tracking_uri())
all_experiments = client.search_experiments()

In [12]:
experiments = client.search_experiments(view_type="ALL")  # includes deleted ones
for exp in experiments:
    print(f"ID={exp.experiment_id} | Name={exp.name} | Stage={exp.lifecycle_stage}")

ID=3 | Name=IRIS classifier: Poisoned | Stage=active
ID=2 | Name=IRIS classifier | Stage=active
ID=1 | Name=OPPE1 stock exp | Stage=active
ID=0 | Name=Default | Stage=active


In [13]:
mlflow.get_tracking_uri()

'http://34.27.101.147:8100'

In [223]:
mlflow.set_experiment('IRIS classifier: Poisoned')

<Experiment: artifact_location='gs://oppe1_winged-quanta-472908-n1/3', creation_time=1763220399280, experiment_id='3', last_update_time=1763220399280, lifecycle_stage='active', name='IRIS classifier: Poisoned', tags={'mlflow.experimentKind': 'custom_model_development'}>

In [126]:
# Restore it if found
if exp is not None and exp.lifecycle_stage == "deleted":
    client.restore_experiment(exp.experiment_id)
    print(f"Restored experiment: {exp.name}")
else:
    print("Experiment not found or already active.")

Experiment not found or already active.


In [15]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score,precision_score, recall_score, f1_score
import joblib
from datetime import datetime
import os

# Load the data
script_dir = script_dir = os.getcwd()
data_path = os.path.join(script_dir, "data")
X_train = pd.read_csv(os.path.join(data_path, "X_train.csv"))
X_test = pd.read_csv(os.path.join(data_path, "X_test.csv"))
y_train = pd.read_csv(os.path.join(data_path, "y_train.csv")).values.ravel()  # flatten
y_test = pd.read_csv(os.path.join(data_path, "y_test.csv")).values.ravel()

In [19]:
def poison_features(X, poison_pct, random_state=42):
    X_poisoned = X.copy()
    np.random.seed(random_state)

    n_rows = len(X)
    n_poison = int(n_rows * (poison_pct / 100))

    # Randomly choose rows to corrupt
    poison_indices = np.random.choice(n_rows, n_poison, replace=False)

    # For each feature column, replace selected rows with random values IN RANGE
    for col in X.columns:
        col_min, col_max = X[col].min(), X[col].max()
        random_values = np.random.uniform(low=col_min, high=col_max, size=n_poison)
        X_poisoned.loc[poison_indices, col] = random_values

    return X_poisoned, poison_indices


In [20]:
def poison_labels(y, poison_indices, random_state=42):
    np.random.seed(random_state)

    y_poisoned = y.copy()

    unique_labels = np.unique(y)
    for idx in poison_indices:
        current_label = y_poisoned[idx]
        choices = [l for l in unique_labels if l != current_label]
        y_poisoned[idx] = np.random.choice(choices)

    return y_poisoned


In [255]:
# Load the data
script_dir = script_dir = os.getcwd()
data_path = os.path.join(script_dir, "data")
X_train = pd.read_csv(os.path.join(data_path, "X_train.csv"))
X_test = pd.read_csv(os.path.join(data_path, "X_test.csv"))
y_train = pd.read_csv(os.path.join(data_path, "y_train.csv")).values.ravel()  # flatten
y_test = pd.read_csv(os.path.join(data_path, "y_test.csv")).values.ravel()

In [256]:
X_train_pos, poison_idx = poison_features(X_train, poison_pct=50)

y_train = poison_labels(y_train, poison_idx)

In [257]:
params = {
    "max_depth": 3,
    "random_state": 42,
    "min_samples_split": 2 
}

In [258]:
# Initialize the Decision Tree classifier with some simple hyperparameters
clf = DecisionTreeClassifier(**params)

# Train the model
clf.fit(X_train, y_train)

# Evaluate
y_pred = clf.predict(X_test)

In [259]:
# Compute metrics
metrics = {
    "accuracy": accuracy_score(y_test, y_pred),
    "precision": precision_score(y_test, y_pred, average="weighted"),
    "recall": recall_score(y_test, y_pred, average="weighted"),
    "f1_score": f1_score(y_test, y_pred, average="weighted")
}

In [260]:
print(metrics)

{'accuracy': 0.6, 'precision': 0.7777777777777778, 'recall': 0.6, 'f1_score': 0.5614589921990302}


In [263]:
with mlflow.start_run():
    mlflow.log_params(params)
    
    mlflow.log_metrics(metrics)
    mlflow.log_metrics(val_metrics)
    
    mlflow.set_tag("Training info","Decision Tree model IRIS data")
    
    signature = infer_signature(X_test[:10], clf.predict(X_test[:10]))
    
    model_info = mlflow.sklearn.log_model(
        sk_model = clf,
        name="iris_model_pos",
        signature = signature,
        input_example = X_train[:10],
        registered_model_name = "IRIS-classifier-dt-pos"
    )

Registered model 'IRIS-classifier-dt-pos' already exists. Creating a new version of this model...
2025/11/15 19:21:00 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: IRIS-classifier-dt-pos, version 4


🏃 View run chill-croc-443 at: http://34.27.101.147:8100/#/experiments/3/runs/bed88edc952a401f939bc048963ba788
🧪 View experiment at: http://34.27.101.147:8100/#/experiments/3


Created version '4' of model 'IRIS-classifier-dt-pos'.


In [3]:
model_name = "IRIS-classifier-dt"
versions = client.search_model_versions(f"name='{model_name}'")

best_version = None
best_accuracy = 0

for v in versions:
    run_id = v.run_id
    run = client.get_run(run_id)
    acc = run.data.metrics.get("accuracy")

    if acc is not None and acc > best_accuracy:
        best_accuracy = acc
        best_version = v

if best_version:
    print(f"Best model version: {best_version.version}")
    print(f"Run ID: {best_version.run_id}")
    print(f"Accuracy: {best_accuracy}")
    print(f"Stage: {best_version.current_stage}")

    # Load the best model directly
    best_model_uri = f"models:/{model_name}/{best_version.version}"
    best_model = mlflow.sklearn.load_model(best_model_uri)
    print(f"Loaded best model from registry: {best_model_uri}")

else:
    print("No versions found for this model.")


Best model version: 3
Run ID: 14b5a5def4ce48daa3dc1c98dfd0ea5e
Accuracy: 1.0
Stage: None


/home/shared_workspace/mlops_wk2/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loaded best model from registry: models:/IRIS-classifier-dt/3


In [261]:
# Load eval data
test_data = pd.read_csv(os.path.join(data_path, "validate.csv")) 
y_test = test_data[['species']].values.ravel()
X_test = test_data.drop('species',axis=1)

# Predict
y_pred = clf.predict(X_test)

# Compute metrics
val_metrics = {
    "val_accuracy": accuracy_score(y_test, y_pred),
    "val_precision": precision_score(y_test, y_pred, average="weighted"),
    "val_recall": recall_score(y_test, y_pred, average="weighted"),
    "val_f1_score": f1_score(y_test, y_pred, average="weighted")
}

In [262]:
print(val_metrics)

{'val_accuracy': 0.6633663366336634, 'val_precision': 0.7994384513078173, 'val_recall': 0.6633663366336634, 'val_f1_score': 0.6285468352660509}
